# Search Data Structures

## Basic Puzzle

 ### Valid moves: Regular (R), Wrap (W), Diagonal (D)
![board](board.jpg)

    - Corners
        - Q0, Q3, Q4, Q7
        - Moves: R, W, D
    - Non-Corners
        - Q1, Q2, Q5, Q6
        - Moves: R

## Puzzle Class
    - Puzzle(tiles, rows, columns)
        - tiles: takes a 1D array of tiles
            - 2X4: [0, 1, 2, 3, 4, 5, 6, 7]
        - rows: # of rows for the puzzle
        - columns: # of columns for the puzzle
        
## Puzzle Fields
    - rows: # of rows
    - columns: # of columns
    - board: current state of the puzzle
    - options: successors generated from current state
    - corners: 4 corners of the board
    
## Puzzle Methods
    - update(new_state)
        - updates the state with a successor
    - board_print(board)
        - board: 1D array to be printed
        - prints a 1D array in grid form to show the board
    - goal()
        - compares current state to the puzzle goal states
    - successor()
        - generates all children from the current state
        - populates the puzzle instance's options with the state successors
            - game = Puzzle([0, 1, 2, 3, 4, 5, 6, 7], 2, 4)
            - game.options is an array of tuples of the form (option, cost)
                - option is a successor state as a 1D array
                - cost is the cost of the move to go from current state to the option state
    - heuristics can run on the successor states in game.options

In [1]:
def board_print(board, columns):
    grid = ''
    for tile in range(len(board)):
        if tile % columns == 0:
            grid = grid + "\n"
        if board[tile] >= 10:
            grid = grid + "  "+ str(board[tile])
        elif board[tile] < 10:
            grid = grid + "   "+ str(board[tile])
    print(grid, "\n")
        
class PuzzleNode:    
    def __init__(self, state, rxc):        
        self.rows = rxc[0]
        
        self.columns = rxc[1]
        
        # 1D array representation
        self.board = state[2].copy()
        
        # state tuple (tile moved, cost, state configuration)
        self.state = state
                
        # board_print(self.board, self.columns)
        
        self.corners = [0, self.columns-1, (len(self.board)-self.columns), (len(self.board)-1) ]
        
        self.successors()
        
    def set_goals(self):
        goal_one = sorted([tile for tile in self.board]).append(0)
        print(goal_one)
        evens = [tile for tile in sorted(self.board) if tile % 2 !=0]
        odds = [tile for tile in sorted(self.board) if tile % 2 == 0]
        goal_two = (evens + odds).append(0)
        print(goal_two)
        return (goal_one, goal_two)
    
    # next_state is a state tuple (tile moved, move cost, new state config )
    def update(self, next_state, *parent):
        self.board = next_state[2]
        self.state = next_state
        self.successors(parent)
        
    # tuple form (tile moved, move cost, new configuration, parent)
    # passing in an optional parent board config will check to make sure it does not generate a parent configuration 
    # back again
    def successors(self, *parent):
        blank =  self.board.index(0)
        self.options = []
        
        # Non-Corner Moves
        if blank not in self.corners:
            # move right if not in the last column
            if ((blank+1) % self.columns) != 0:
                self.move(blank, 'r')
            
            # move left if not in the first column ()
            if (blank % self.columns) != 0:
                self.move(blank, 'l')
            
            # move down  if (blank + column) <= length (not in the last row)
            if (blank+self.columns) <= len(self.board):
                self.move(blank, 'd')
                
            # move up if (blank - column) >= 0 (not in the first row)
            if (blank-self.columns) >= 0:
                self.move(blank, 'u')
                
        # Corner Moves  
        elif blank in self.corners:
            # Top-Left (0)
            if self.corner(blank) == 0:
                self.move(blank, 'r')
                
                self.move(blank, 'd')
                
                if self.rows > 2:
                    self.move(blank, 'wu')
                    
                self.move(blank, 'wl')
                
                # diagonal to BR
                diagonal_1 = self.board.copy()
                next_tile = len(self.board)-1
                diagonal_1[blank] = diagonal_1[next_tile]
                diagonal_1[next_tile] = 0
                self.options.append((diagonal_1[blank], 3, diagonal_1))
                # board_print(diagonal_1, self.columns)
                
                # diagonal down (down + 1)
                diagonal_2 = self.board.copy()
                next_tile = self.columns+1
                diagonal_2[blank] = diagonal_2[next_tile]
                diagonal_2[next_tile] = 0
                self.options.append((diagonal_2[blank], 3, diagonal_2))
                # board_print(diagonal_2, self.columns)
                
            # Top-Right (1)
            if self.corner(blank) == 1:
                self.move(blank, 'l')
                
                self.move(blank, 'd')
                
                if self.rows > 2:
                    self.move(blank, 'wu')
                    
                self.move(blank, 'wr')
                
                # diagonal to BL
                diagonal_1 = self.board.copy()
                next_tile = len(self.board)-self.columns
                diagonal_1[blank] = diagonal_1[next_tile]
                diagonal_1[next_tile] = 0
                self.options.append((diagonal_1[blank], 3, diagonal_1))
                # board_print(diagonal_1, self.columns)
                
                # diagonal down (down - 1)
                diagonal_2 = self.board.copy()
                next_tile = blank+self.columns-1
                diagonal_2[blank] = diagonal_2[next_tile]
                diagonal_2[next_tile] = 0
                self.options.append((diagonal_2[blank], 3, diagonal_2))
                # board_print(diagonal_2, self.columns)
                
            # Bottom-Left (2)
            if self.corner(blank) == 2:
                self.move(blank, 'r')
                
                self.move(blank, 'u')
                
                if self.rows > 2:
                    self.move(blank, 'wd')
                    
                self.move(blank, 'wl')
                
                # diagonal to TR
                diagonal_1 = self.board.copy()
                next_tile = self.columns-1
                diagonal_1[blank] = diagonal_1[next_tile]
                diagonal_1[next_tile] = 0
                self.options.append((diagonal_1[blank], 3, diagonal_1))
                # board_print(diagonal_1, self.columns)
                
                # diagonal up (up + 1)
                diagonal_2 = self.board.copy()
                next_tile = blank-self.columns+1
                diagonal_2[blank] = diagonal_2[next_tile]
                diagonal_2[next_tile] = 0
                self.options.append((diagonal_2[blank], 3, diagonal_2))
                # board_print(diagonal_2, self.columns)
                
            # Bottom-Right (3)
            if self.corner(blank) == 3:
                self.move(blank, 'l')
                
                self.move(blank, 'u')
                
                if self.rows > 2:
                    self.move(blank, 'wd')
                    
                self.move(blank, 'wr')
                
                # diagonal to TL
                diagonal_1 = self.board.copy()
                diagonal_1[blank] = diagonal_1[0]
                diagonal_1[0] = 0
                self.options.append((diagonal_1[blank], 3, diagonal_1))
                # board_print(diagonal_1, self.columns)
                
                # diagonal up (up - 1)
                diagonal_2 = self.board.copy()
                next_tile = blank-self.columns-1
                diagonal_2[blank] = diagonal_2[next_tile]
                diagonal_2[next_tile] = 0
                self.options.append((diagonal_2[blank], 3, diagonal_2))
                # board_print(diagonal_2, self.columns)
                
            if parent is not None:
                for x in self.options:
                    if x[2]  == parent:
                        self.options.remove(x)
   
            
    def corner(self, blank):
            # Top-left corner
            if blank == 0:
                return 0
            # Top-right corner
            if blank == (self.columns-1):
                return 1
            # Bottom-left corner
            if blank == (len(self.board)-self.columns):
                return 2
            # Bottom-right corner
            if blank == (len(self.board)-1):
                return 3
        
    def move(self, blank, direction):
        # Move right (r)
        if direction == 'r':
            right = self.board.copy()
            right[blank] = right[blank+1]
            right[blank+1] = 0
            self.options.append((right[blank], 1, right))
            # board_print(right, self.columns)
        
        # Move left (l)
        if direction == 'l':
            left = self.board.copy()
            left[blank] = left[blank-1]
            left[blank-1] = 0
            self.options.append((left[blank], 1, left))
            # board_print(left, self.columns)
        
        # Move down (d)
        if direction == 'd':
            down = self.board.copy()
            down[blank] = down[blank+self.columns]
            down[blank+self.columns] = 0
            self.options.append((down[blank], 1, down))
            # board_print(down, self.columns)
        
        # Move up (u)
        if direction == 'u':
            up = self.board.copy()
            up[blank] = up[blank-self.columns]
            up[blank-self.columns] = 0
            self.options.append((up[blank], 1, up))
            # board_print(up, self.columns)
        
        # Wrap right (wr)
        if direction == 'wr':
            wrap_right = self.board.copy()
            wrap_right[blank] = wrap_right[blank-self.columns+1]
            wrap_right[blank-self.columns+1] = 0
            self.options.append((wrap_right[blank], 2, wrap_right))
            # board_print(wrap_right, self.columns)
            
        # Wrap left (wl)
        if direction == 'wl':
            wrap_left = self.board.copy()
            wrap_left[blank] = wrap_left[blank+self.columns-1]
            wrap_left[blank+self.columns-1] = 0
            self.options.append((wrap_left[blank], 2, wrap_left))
            # board_print(wrap_left, self.columns)
            
        # Wrap down (wd)
        if direction == 'wd':
            wrap_down = self.board.copy()
            next_tile = blank-(self.columns * (self.rows-1))
            wrap_down[blank] = wrap_down[next_tile]
            wrap_down[next_tile] = 0
            self.options.append((wrap_down[blank], 2, wrap_down))
            # board_print(wrap_down, self.columns)
            
        # Wrap up (wu)
        if direction == 'wu':
            wrap_up = self.board.copy()
            next_tile = blank+(self.columns * (self.rows-1))
            wrap_up[blank] = wrap_up[next_tile]
            wrap_up[next_tile] = 0
            self.options.append((wrap_up[blank], 2, wrap_up))
            # board_print(wrap_up, self.columns)
        

# Search Setup

In [2]:
b2x4 = [0, 1, 2, 3, 4, 5, 6, 7]
b3x4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
b4x4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
b5x4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
b5x5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

# all samples are 2X4
sample1 = [3, 0, 1, 4, 2, 6, 5, 7]
sample2 = [6, 3, 4, 7, 1, 2, 5, 0]
sample3 = [1, 0, 3, 6, 5, 2, 7, 4]

sample_puzzles = [sample1, sample2, sample3]

## Read Input Puzzles

In [3]:
filename = "samplePuzzles.txt"

with open(filename) as f:
    content = f.readlines()

puzzles = []    
for line in content:
    puzzle_str = line.replace("\n", "").split(' ')
    puzzle = [int(x) for x in puzzle_str]
    puzzles.append(puzzle)
    
for puzzle in puzzles:
    print(puzzle)

[3, 0, 1, 4, 2, 6, 5, 7]
[6, 3, 4, 7, 1, 2, 5, 0]
[1, 0, 3, 6, 5, 2, 7, 4]


## Generate Solution Path from Goal State Node

In [4]:
def solution(end_node, tree):
    solved = []
    n = end_node
    while n['parent'] is not None:
        solved.append(n)
        n = tree[n['parent']]
    solved.append(n)
    return solved

## Define Goal States and Initial Node

In [5]:
goal_states = ([1, 2, 3, 4, 5, 6, 7, 0], [1, 3, 5, 7, 2, 4, 6, 0])
goal1 = goal_states[0]
goal2 = goal_states[1]
rows_by_columns = (2, 4)
import time

## Define g(n)
- the cost of solution path from n to start node

In [6]:
def g(n, tree):
    cost = n['current'][1]
    i = n
    while i['parent'] is not None:
        cost = cost + i['current'][1]
        i = tree[i['parent']]
    return cost

# Uniform Cost Search

In [7]:
# puzzle_num = -1 indicates this is being run for analytics
# with 50 random files and should not write to file

def runUniformCostSearch(puzzle, puzzle_num = -1):
    
    # ---analysis variables--- # 
    
    solution_path_length = 0
    search_path_length = 0
    no_solutions = 0
    cost = 0
    execution_time = 0
    
    # ---analysis variables--- #
    
    initial_problem = PuzzleNode((0, 0, puzzle), rows_by_columns)
    initialNode = ({ 'parent': None, 'current': initial_problem.state }, 0)
    
    search_file = "./output/puzzle_" + str(puzzle_num) + "/" + str(puzzle_num) + "_ucs-search.txt"
    solution_file = "./output/puzzle_" + str(puzzle_num) + "/" + str(puzzle_num) + "_ucs-solution.txt"
                
    if puzzle_num != -1:
        open(search_file, 'w').close()

    # Initialize open list with s
    open_list = [initialNode]

    # Initialize closed list as empty
    closed_list = []

    tree = {}
    j = 0

    start_time = time.process_time()

    current_problem = initial_problem

    # Loop while the puzzle is not in either goal state
    while True:
        
#         Check time limit
        if (time.process_time() - start_time) > 60:
        
            solution_path_length = 0
            search_path_length = 0
            no_solutions = 1
            cost = 0
  
            if puzzle_num != -1:
                f = open(solution_file, 'w')
                f.write('No Solution')
                f.close()

                f = open(search_file, 'w')
                f.write('No Solution')
                f.close()
            
            
            print("Search failed: 60 second time limit reached")
            break
        
        # Check if open list is empty
        if not open_list:
            print("Search failed: Open list is empty..")
            break
        else:
            # get the next state S
            s = open_list.pop()[0]
            
            search_path_length += 1
            
            if puzzle_num != -1:
                f = open(search_file, "a")
                line = "_".join(['0', str(g(s, tree)), '0'] + [str(x) for x in s['current'][2]])
                f.write(line + '\n')
            
            closed_list.append(s)

            # place s in the tree
            tree[j] = s

            current_problem.update(s['current'])

            # check if S is in either goal state
            if s['current'][2] in goal_states:
                print("Search complete: Found goal state")

                # extract solution
                solution_path = solution(s, tree)
                
                solution_path.reverse()
                
                if puzzle_num != -1:
                    open(solution_file, 'w').close()
                
                    f = open(solution_file, "a")
                

                for node in solution_path:
                    
                    solution_path_length += 1
                    cost += node['current'][1]
                    
                    if puzzle_num != -1:
                        entry_arr = [str(x) for x in node['current'][:2]] + [str(x) for x in node['current'][2]]
                        line = '_'.join(entry_arr)
                    
                        f.write(line + '\n')

#                     print(node)
#                     board_print(node['current'][2], rows_by_columns[1])

                if puzzle_num != -1:
                    f.close()

                break

            # not in goal state
            else:
                closed_states = [closed['current'][2] for closed in closed_list]
                open_states = [opened[0]['current'][2] for opened in open_list]

                # add children to open list
                for child in current_problem.options:
                    s_child = {'parent': j, 'current': child}
                    
                    if child[2] not in closed_states and child[2] not in open_states:
                    
                        open_list.append((s_child, g(s_child, tree)))

                    # if the board configuration is already in the open[], choose lowest cost one
                    elif child[2] not in closed_states and child[2] in open_states:
                        #find the duplicate in open list
                        for i in range(len(open_list)):
                            if child[2] == open_list[i][0]['current'][2]:
                                new_g = g(s_child, tree) 
                                #compare g(n)'s
                                if open_list[i][1] > new_g:
                                    open_list.remove(open_list[i])
                                    open_list.append((s_child, new_g))
                                #else, we do nothing... we keep the version that is already in open[]

                # sort open list by g(n)
                open_list = sorted(open_list, key=lambda n: n[1], reverse=True)

                j = j + 1     

    execution_time = (time.process_time() - start_time)
    print("--- %s seconds ---" % (execution_time) )
    
    if puzzle_num != -1:
        with open(solution_file, 'a') as output_file:
            output_file.write( 'cost: {}, execution-time: {}'.format(cost, execution_time) )
            
    return solution_path_length, search_path_length, no_solutions, cost, execution_time

In [8]:
for i, puzzle in enumerate(sample_puzzles):
    runUniformCostSearch(puzzle, i)

Search failed: 60 second time limit reached
--- 60.01624 seconds ---
Search complete: Found goal state
--- 18.192300000000003 seconds ---
Search complete: Found goal state
--- 0.3337409999999892 seconds ---


# Greedy Best-First Search

In [9]:
# puzzle_num = -1 indicates this is being run for analytics
# with 50 random files and should not write to file

def runBestFirstSearch(puzzle, h, puzzle_num = -1):
    
    # ---analysis variables--- # 
    
    solution_path_length = 0
    search_path_length = 0
    no_solutions = 0
    cost = 0
    execution_time = 0
    
    # ---analysis variables--- #
    
    initial_problem = PuzzleNode((0, 0, puzzle), rows_by_columns)
    initialNode = ({ 'parent': None, 'current': initial_problem.state }, 0)
    
    # these 2 are the same
    #print(current_problem.state[2])
    #print(current_problem.board)
    
    solution_file = './output/puzzle_{}/{}_gbfs-{}_solution.txt'.format(puzzle_num, puzzle_num, h.__name__)
    search_file = './output/puzzle_{}/{}_gbfs-{}_search.txt'.format(puzzle_num, puzzle_num, h.__name__)
                
    if puzzle_num != -1:
        open(search_file, 'w').close()
    

    # Initialize open list with s
    open_list = [initialNode]
    

    # Initialize closed list as empty
    closed_list = []

    tree = {}
    j = 0

    start_time = time.process_time()
    
    current_problem = initial_problem

    # Loop while the puzzle is not in either goal state
    while True:
        # Check if open list is empty
        if not open_list:
            print("Search failed: Open list is empty..")
            break
        else:
            # get the next state S
            s = open_list.pop()[0]
            
            search_path_length += 1
            
            if puzzle_num != -1:
                f = open(search_file, "a")
                line = "_".join(['0','0', str(h(s['current'][2]))] + [str(x) for x in s['current'][2]])
                f.write(line + '\n')
            
            closed_list.append(s) 

            # place s in the tree
            tree[j] = s

            current_problem.update(s['current']) # update state, board, and successors() to access .options

            # check if S is in either goal state
            if s['current'][2] in goal_states:
                print("Search complete: Found goal state")

                # extract solution
                solution_path = solution(s, tree)
                
                solution_path.reverse()
                
                if puzzle_num != -1:
                    open(solution_file, 'w').close()
                    f = open(solution_file, "a")
                

                for node in solution_path:
                    solution_path_length += 1
                    cost += node['current'][1]
                    
                    if puzzle_num != -1:
                        entry_arr = [str(x) for x in node['current'][:2]] + [str(x) for x in node['current'][2]]
                        line = '_'.join(entry_arr)
                    
                        f.write(line + '\n')
                
#                     print(node)
#                     board_print(node['current'][2], rows_by_columns[1])

                if puzzle_num != -1:
                    f.close()
                
                break

            # not in goal state
            else:
                closed_states = [closed['current'][2] for closed in closed_list]
                open_states = [opened[0]['current'][2] for opened in open_list]

                # add children to open list: Use node.options
                for child in current_problem.options:
                    if child[2] not in closed_states and child[2] not in open_states:
                        s_child = {'parent': j, 'current': child} # successor and parent
                        open_list.append((s_child, h(child[2]))) # here call h(n) to assign cost to successor s

                # sort open list by h(n)
                open_list = sorted(open_list, key=lambda n: n[1], reverse=True) 

                j = j + 1   

    execution_time = (time.process_time() - start_time)
    print("--- %s seconds ---" % (execution_time) )
    
    if puzzle_num != -1:
        with open(solution_file, 'a') as output_file:
            output_file.write( 'cost: {}, execution-time: {}'.format(cost, execution_time))
        
    
    return solution_path_length, search_path_length, no_solutions, cost, execution_time

## Define h1(n)
- h1(n) is an estimate of the cost from node n to the goal
- It is the Hamming distance: 
- count number of tiles out of place when comparing s_child to both goals

In [10]:
def h1(s_board):
    cost1 = 0
    cost2 = 0
    #check goal1
    for i in range(len(goal1)):
        if s_board[i] != goal1[i]:
            cost1+=1   
    #check goal2
    for k in range(len(goal2)):
        if s_board[k] != goal2[k]:
            cost2+=1
            
    #return average of cost to goal1 and goal2
    return min(cost1, cost2)

## Define h2(n)
- h2(n) is an estimate of the cost from node n to the goal
- It is the Manhattan distance: 
- sum distances by which the tiles are out of place when comparing s_child to both goals-

In [11]:
def h2(s_board):
    cost1 = 0
    cost2 = 0
    #check goal1
    for i in range(len(goal1)):
        for j in range(len(goal1)):
            if s_board[i] == goal1[j]:
                cost1 = cost1 + abs(i-j)
            
    #check goal2
    for k in range(len(goal2)):
        for m in range(len(goal2)):
            if s_board[k] == goal2[m]:
                cost2 = cost2 + abs(k-m)
            
    #return average of cost to goal1 and goal2
    return min(cost1, cost2)

## Define h3(n)
- h3(n) is an estimate of the cost from node n to the goal
- It is the Sum of permutation inversions

In [12]:
def h3(s_board):
    cost1 = 0
    cost2 = 0
    #check goal1
    for i in range(len(s_board)):
        for j in range(i, len(s_board)):
            if (s_board[j] < s_board[i]) and (s_board[j] != 0):
                cost1 += 1       
    #check goal2
    for i in range(len(s_board)):
        for j in range(i, len(s_board)):
            #for 0, anything on its right is +1
            if s_board[i] == 0:
                if s_board[j] != 0:
                    cost2 += 1
            #even
            elif s_board[i]%2 == 0:
                # all odds go on lhs of evens
                if (s_board[j]%2 != 0):
                    cost2 += 1
                # order evens except the 0    
                elif (s_board[j]%2 == 0) and (s_board[j] < s_board[i]) and (s_board[j]!=0):
                    cost2 += 1     
            #odd
            else:
                # no even goes on its left side, only checks for odds
                if(s_board[j]%2 != 0):
                    if s_board[j] < s_board[i]:
                        cost2 += 1
                        
    #return average of cost to goal1 and goal2
    return (cost1+cost2)/2

## Greedy Best-First Search h1(n)

In [13]:
h_funcs = [h1, h2, h3]
    
for h in h_funcs:
    for i, puzzle in enumerate(sample_puzzles):
        runBestFirstSearch(puzzle, h, i)

Search complete: Found goal state
--- 0.016045000000005416 seconds ---
Search complete: Found goal state
--- 0.0023130000000008977 seconds ---
Search complete: Found goal state
--- 0.008637999999990598 seconds ---
Search complete: Found goal state
--- 0.37418799999998953 seconds ---
Search complete: Found goal state
--- 0.040537999999997965 seconds ---
Search complete: Found goal state
--- 0.008969000000007554 seconds ---
Search complete: Found goal state
--- 0.047295000000005416 seconds ---
Search complete: Found goal state
--- 0.039610999999993624 seconds ---
Search complete: Found goal state
--- 0.008228000000002567 seconds ---



## Algorithm A

In [14]:
# puzzle_num = -1 indicates this is being run for analytics
# with 50 random files and should not write to file

def runAlgoA(puzzle, h, puzzle_num = -1):
    
    # ---analysis variables--- # 
    
    solution_path_length = 0
    search_path_length = 0
    no_solutions = 0
    cost = 0
    execution_time = 0
    
    # ---analysis variables--- #
    
    def f(n, tree):
        return h(n['current'][2]) + g(n, tree)

    initial_problem = PuzzleNode((0, 0, puzzle), rows_by_columns)
    initialNode = ({ 'parent': None, 'current': initial_problem.state }, 0)
    
    
    solution_file = './output/puzzle_{}/{}_astar-{}_solution.txt'.format(puzzle_num, puzzle_num, h.__name__)
    search_file = './output/puzzle_{}/{}_astar-{}_search.txt'.format(puzzle_num, puzzle_num, h.__name__)
    
    
    if puzzle_num != -1:
        open(search_file, 'w').close()
    

    # Initialize open list with s
    open_list = [initialNode]
    

    # Initialize closed list as empty
    closed_list = []

    cost_sum = 0
    tree = {}
    j = 0

    start_time = time.process_time()
    
    current_problem = initial_problem
    
    # Loop while the puzzle is not in either goal state
    while True:
        # Check if open list is empty
        if not open_list:
            print("Search failed: Open list is empty..")
            break
        else:
            # get the next state S
            s = open_list.pop()[0]
            f_cost = f(s, tree)
            
            search_path_length += 1
            
            if puzzle_num != -1:
                
                outFile = open(search_file, "a")
            
                h_cost = h(s['current'][2])
                g_cost = g(s, tree)
            
                entry_arr = [str(f_cost),str(g_cost), str(h_cost)] +  [str(x) for x in s['current'][2]]
                line = "_".join(entry_arr)

                outFile.write(line + '\n')
            
            closed_list.append((s, f_cost)) 

            # place s in the tree
            tree[j] = s

            current_problem.update(s['current']) # update state, board, and successors() to access .options

            # check if S is in either goal state
            if s['current'][2] in goal_states:
                print("Search complete: Found goal state")

                #extract solution
                solution_path = solution(s, tree)
                
                solution_path.reverse()
                
                if puzzle_num != -1:
                    open(solution_file, 'w').close()
                    outFile = open(solution_file, "a")
                

                for node in solution_path:
                    solution_path_length += 1
                    cost += node['current'][1]
                    
                    if puzzle_num != -1:
                        entry_arr = [str(x) for x in node['current'][:2]] + [str(x) for x in node['current'][2]]
                        line = '_'.join(entry_arr)

                        outFile.write(line + '\n')

                    print(node)
                    board_print(node['current'][2], rows_by_columns[1])

                if puzzle_num != -1:
                    outFile.close()
                
                break

            # not in goal state
            else:
                closed_states = [closed[0]['current'][2] for closed in closed_list]
                open_states = [opened[0]['current'][2] for opened in open_list]


                # add children to open list
                for child in current_problem.options:
                    s_child = {'parent': j, 'current': child}
                    new_f = f(s_child, tree) 
                    
                    if child[2] not in closed_states and child[2] not in open_states:    
                        open_list.append((s_child, new_f))

                        
                    elif child[2] in closed_states:
                        #find the duplicate in closed list
                        for i in range(len(closed_list)):
                            if child[2] == closed_list[i][0]['current'][2]:
                                if closed_list[i][1] > new_f:
                                    closed_list.remove(closed_list[i])
                                    open_list.append((s_child, new_f))
                                    break;
                                # else, we do nothing... we keep the version that is already in closed[]
                    
                    # if the board configuration is already in the open[], choose lowest cost one
                    elif child[2] in open_states:
                        #find the duplicate in open list
                        for i in range(len(open_list)):
                            if child[2] == open_list[i][0]['current'][2]:
                                #compare f(n)'s
                                if open_list[i][1] > new_f:
                                    open_list.remove(open_list[i])
                                    open_list.append((s_child, new_f))
                                # else, we do nothing... we keep the version that is already in open[]
                                
            

                # sort open list by f(n)
                open_list = sorted(open_list, key=lambda n: n[1], reverse=True)

                j = j + 1      

    execution_time = (time.process_time() - start_time)
    print("--- %s seconds ---" % (execution_time) )
    
    if puzzle_num != -1:
        with open(solution_file, 'a') as output_file:
            output_file.write( 'cost: {}, execution-time: {}'.format(cost, execution_time))
        
    return solution_path_length, search_path_length, no_solutions, cost, execution_time

In [15]:
def dummyH(s):
    return 0

h_funcs = [h1, h2, h3]
    
for h in h_funcs:
    for i, puzzle in enumerate(sample_puzzles):
        runAlgoA(puzzle, h, i)

Search complete: Found goal state
{'parent': None, 'current': (0, 0, [3, 0, 1, 4, 2, 6, 5, 7])}

   3   0   1   4
   2   6   5   7 

{'parent': 0, 'current': (3, 1, [0, 3, 1, 4, 2, 6, 5, 7])}

   0   3   1   4
   2   6   5   7 

{'parent': 1, 'current': (2, 1, [2, 3, 1, 4, 0, 6, 5, 7])}

   2   3   1   4
   0   6   5   7 

{'parent': 6, 'current': (4, 3, [2, 3, 1, 0, 4, 6, 5, 7])}

   2   3   1   0
   4   6   5   7 

{'parent': 536, 'current': (1, 1, [2, 3, 0, 1, 4, 6, 5, 7])}

   2   3   0   1
   4   6   5   7 

{'parent': 542, 'current': (5, 1, [2, 3, 5, 1, 4, 6, 0, 7])}

   2   3   5   1
   4   6   0   7 

{'parent': 543, 'current': (6, 1, [2, 3, 5, 1, 4, 0, 6, 7])}

   2   3   5   1
   4   0   6   7 

{'parent': 544, 'current': (4, 1, [2, 3, 5, 1, 0, 4, 6, 7])}

   2   3   5   1
   0   4   6   7 

{'parent': 545, 'current': (2, 1, [0, 3, 5, 1, 2, 4, 6, 7])}

   0   3   5   1
   2   4   6   7 

{'parent': 546, 'current': (1, 2, [1, 3, 5, 0, 2, 4, 6, 7])}

   1   3   5   0
   2   4  

## Run Analysis on Random Puzzles

### Generate Random Puzzles

In [16]:
import random
random_puzzles = []
for i in range(50):
    newPuzzle = [0, 1, 2, 3, 4, 5, 6, 7]
    random.shuffle(newPuzzle)
    random_puzzles.append(newPuzzle)
    
    # todo write to files

In [20]:
analysis_file = "./output/analysis.txt"

# clear analysis file
open(analysis_file, 'w').close()

outFile = open(analysis_file, "a")

def writeStats(solution_path_length, search_path_length, no_solutions, cost, execution_time):
    num_solved = 50 - no_solutions
    outFile.write('Total Length of Solution Path:fsdaf {}\n'.format(solution_path_length))
    outFile.write('Average Length of Solution Path: {}\n'.format(solution_path_length / num_solved))
    outFile.write('Total Length of Search Path: {}\n'.format(search_path_length))
    outFile.write('Average Length of Search Path: {}\n'.format(search_path_length / num_solved))
    outFile.write('Total No Solutions: {}\n'.format(no_solutions))
    outFile.write('Total Cost of Solution Path: {}\n'.format(cost))
    outFile.write('Average Cost of Solution Path: {}\n'.format(cost / num_solved))
    outFile.write('Total Execution Time: {}\n'.format(execution_time))
    outFile.write('Average Execution Time: {}\n'.format(execution_time / num_solved))


# UNIFORM COST    
    
# totals
solution_path_length = 0
search_path_length = 0
no_solutions = 0
cost = 0
execution_time = 0


# for puzzle in random_puzzles:
    
#     stats = runUniformCostSearch(puzzle)
    
#     solution_path_length += stats[0]
#     search_path_length += stats[1]
#     no_solutions += stats[2]
#     cost += stats[3]
#     execution_time += stats[4]

# outFile.write(' Uniform Cost:')
# writeStats(solution_path_length, search_path_length, no_solutions, cost, execution_time)
# outFile.write('\n')


# GBFS  

for h in h_funcs:
    
    solution_path_length = 0
    search_path_length = 0
    no_solutions = 0
    cost = 0
    execution_time = 0
    
    for puzzle in random_puzzles:

        stats = runBestFirstSearch(puzzle, h)

        solution_path_length += stats[0]
        search_path_length += stats[1]
        no_solutions += stats[2]
        cost += stats[3]
        execution_time += stats[4]

    outFile.write('Greedy Best First {}:'.format(h.__name__))
    writeStats(solution_path_length, search_path_length, no_solutions, cost, execution_time)
    outFile.write('\n')
    
    
# A*  

# for h in h_funcs:
    
#     solution_path_length = 0
#     search_path_length = 0
#     no_solutions = 0
#     cost = 0
#     execution_time = 0
    
#     for puzzle in random_puzzles:

#         stats = runAlgoA(puzzle, h)

#         solution_path_length += stats[0]
#         search_path_length += stats[1]
#         no_solutions += stats[2]
#         cost += stats[3]
#         execution_time += stats[4]

#     outFile.write('A Star {}:'.format(h.__name__))
#     writeStats(solution_path_length, search_path_length, no_solutions, cost, execution_time)
#     outFile.write('\n')

Search complete: Found goal state
--- 0.001356000000214408 seconds ---
Search complete: Found goal state
--- 0.0019960000004175527 seconds ---
Search complete: Found goal state
--- 0.0007810000001882145 seconds ---
Search complete: Found goal state
--- 0.0016369999998460116 seconds ---
Search complete: Found goal state
--- 0.0011359999998603598 seconds ---
Search complete: Found goal state
--- 0.0028269999997974082 seconds ---
Search complete: Found goal state
--- 0.003135999999813066 seconds ---
Search complete: Found goal state
--- 0.004650000000310683 seconds ---
Search complete: Found goal state
--- 0.0026009999996858824 seconds ---
Search complete: Found goal state
--- 0.0021320000000741857 seconds ---
Search complete: Found goal state
--- 0.002466999999796826 seconds ---
Search complete: Found goal state
--- 0.0014639999999417341 seconds ---
Search complete: Found goal state
--- 0.0006100000000515138 seconds ---
Search complete: Found goal state
--- 0.000766000000112399 seconds -

Search complete: Found goal state
--- 0.051543000000037864 seconds ---
Search complete: Found goal state
--- 0.03075300000000425 seconds ---
Search complete: Found goal state
--- 0.029408999999759544 seconds ---
Search complete: Found goal state
--- 0.04642499999999927 seconds ---
Search complete: Found goal state
--- 0.036731000000145286 seconds ---
Search complete: Found goal state
--- 0.02731500000027154 seconds ---
Search complete: Found goal state
--- 0.05067300000018804 seconds ---
Search complete: Found goal state
--- 0.0019629999997050618 seconds ---
Search complete: Found goal state
--- 0.07092699999975594 seconds ---
Search complete: Found goal state
--- 0.031234000000040396 seconds ---
Search complete: Found goal state
--- 0.04046999999991385 seconds ---
Search complete: Found goal state
--- 0.013284000000112428 seconds ---
Search complete: Found goal state
--- 0.01597600000013699 seconds ---
Search complete: Found goal state
--- 0.011045000000194705 seconds ---
Search compl